# Sketch-Buster 9000 
### Even Though I have commited this code a couple of times, the results does not seem to update unfortunately, and I don't know why. It runs fine sometimes in the Interactive Session however

By me and with help from some nice public kernels around. 

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import ast
sns.set_palette(sns.color_palette('copper', 20))
from datetime import date, timedelta



# Exercise 1
Counting classes and amount of data
Takes a long time to run, but it prints: 49.707.579 total counts
340 classes 

In [ ]:
import pandas as pd
import os, os.path
import numpy as np
trainPath = '../input/train_simplified/'
trainClasses = os.listdir(trainPath)
amountClasses = len([name for name in trainClasses])
print('Amount of classes: ' + str(amountClasses))
"""
totalCount = 0
for i in range(amountClasses):
    df = pd.read_csv(trainPath + trainClasses[i])
    count = len(df.index)
    totalCount += count
    print('Progress: ' + str(i/amountClasses))
    
print('Total Count of classes:' + str(totalCount) + 'Count per class: ' + str(count)) 
Takes a long time to run, but it prints: 49.707.579 total counts and 116677 counts per class
This is not correct (as 116677*340 != 49707579) and must mean that the data is not evenly distributed between classes, so the question
"how many images per class" will vary depending on what class we're talking about"""

In [ ]:
import gc
gc.collect()


Showing some example images down below: (takes a long time to process)

In [ ]:
import matplotlib.pyplot as plt
import ast
import json


for i in range(0, 8):
    df = pd.read_csv(trainPath + trainClasses[i])
    df['timestamp'] = pd.to_datetime(df.timestamp)
    df.sort_values(by='timestamp', ascending=False)[-20:]
    df['drawing'] = df['drawing'].apply(json.loads)
    df.head()
    
    n = 10
    j = 0
    fig, axs = plt.subplots(nrows=5, ncols=4, sharex=True, sharey=True)
    for j, drawing in enumerate(df.drawing):
        if j//5 == 5: break
        ax = axs[j // 5, j % 4]
        for x, y in drawing:
            ax.plot(x, -np.array(y), lw = 3);
        ax.axis('off')
        
    plt.show()



# Considerations about data distributions:

1. Drawings are placed differently on the canvas, meaning we should augment some data to train the network to handle translations, scales and rotations
2. Data is not evenly distributed in the classes so we should avoid overfitting to the classes that are overrepresented by using regularzation
3. Colours are used differently in each sample, one should consider converting to B/W to only get the shape. This has the drawback that colour sometimes emphasises what we draw and then we cant use that cue anymore.
4. We have two types of data: simplified and raw. Raw will contain more information, but also noise. Simple will be much faster to train and still have the important and sensible parts
of the drawing. Maybe using a combination of the two will be a good idea to increase accuracy? 


## MY NETWORK: 
I have chosen to convert the drawing to black and white images using the simplified data. This should help with training time and remove any noise there might have been introduced in the raw data. 
The way I do it is starting with a blank canvas and then draw a line from point to point. This means that the line will also be straight and the edges will be sharp. 
To speed up this process I use enumerates so that I can parallel the drawing process. 
I only use the drawings that has been recognized, as the other drawings would probably be invalid. 

The training data is gathered in one large array that is then shuffled. 
I am splitting up my data to that I use 10 percent of it for validation and the rest for training the neural network 




# Exercise 2 

In [ ]:
#%% import
import os
from glob import glob
import re
import ast
import numpy as np 
import pandas as pd
import json
from PIL import Image, ImageDraw 
from tqdm import tqdm
from dask import bag

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LeakyReLU, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

#%% set label dictionary and params
classfiles = os.listdir('../input/train_simplified/')
numstonames = {i: v[:-4].replace(" ", "_") for i, v in enumerate(classfiles)} #adds underscores

num_classes = 340    #340 max 
imheight, imwidth = 32, 32  
ims_per_class = 2000  #max - can always be increased later if we want to train with more images

# faster conversion function since we are using the simplified data
def draw_it(strokes):
    image = Image.new("P", (256,256), color=255)
    image_draw = ImageDraw.Draw(image)
    for stroke in json.loads(strokes):
        for i in range(len(stroke[0])-1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=0, width=5)
    image = image.resize((imheight, imwidth))
    return np.array(image)/255.

#%% get train arrays from the zip file
train_grand = []
class_paths = glob('../input/train_simplified/*.csv')
for i,c in enumerate(tqdm(class_paths[0: num_classes])): #Basically saying that we want to run this 2000 times for 340 classess
    train = pd.read_csv(c, usecols=['drawing', 'recognized'], nrows=ims_per_class*5//4)
    train = train[train.recognized == True].head(ims_per_class)
    imagebag = bag.from_sequence(train.drawing.values).map(draw_it) 
    trainarray = np.array(imagebag.compute())  # PARALLELIZING USING DASK.BAG.COMPUTE (essentially converting to np array)
    trainarray = np.reshape(trainarray, (ims_per_class, -1))    
    labelarray = np.full((train.shape[0], 1), i)
    trainarray = np.concatenate((labelarray, trainarray), axis=1)
    train_grand.append(trainarray)
    
train_grand = np.array([train_grand.pop() for i in np.arange(num_classes)]) #less memory than np.concatenate
train_grand = train_grand.reshape((-1, (imheight*imwidth+1)))

del trainarray # Clearing some memory right here
del train

# splitting up training data
valfrac = 0.1  #using 10 percent for validation
cutpt = int(valfrac * train_grand.shape[0]) #Which means we must use 90% of the data 

np.random.shuffle(train_grand) #shuffling the data around
y_train, X_train = train_grand[cutpt: , 0], train_grand[cutpt: , 1:]  #Splitting it up into two arrays; 612000 instances of 340 classes, and 612000 instances of 32x32 images
y_val, X_val = train_grand[0:cutpt, 0], train_grand[0:cutpt, 1:] #validation set is recognized==True

del train_grand #Removing the large grand array for memory management

y_train = keras.utils.to_categorical(y_train, num_classes) #y contain the possible classes (output)
X_train = X_train.reshape(X_train.shape[0], imheight, imwidth, 1) #x is the image itself (input)
y_val = keras.utils.to_categorical(y_val, num_classes) #same
X_val = X_val.reshape(X_val.shape[0], imheight, imwidth, 1)

print(y_train.shape, "\n",
      X_train.shape, "\n",
      y_val.shape, "\n",
      X_val.shape)

I have created my model more or less from trial and error. This means setting the epoch low to a start, running it, changing a variable, and the running it again.
I create my sequential model. I use a standard CNN and I try not to make it too deep, as the drawings should be relatively simple when they have been drawn from skratch in B/W. It also helps with training time when it is not too deep.
I use the activation function RELU , as it turned out that it was the one that performed the best. I have tried sigmoid, linear and leakyRELU, however leakyRELU I found to give better initial results, but performing worse in the higher epochs. 

I use max pooling after each convolutional layer, as I follow the standard CNN architecture. I have experiemented with the size of the max pooling kernel, but 2x2  performs best. This might be because of data is sparse in the drawings, since they only consist of thin lines and sharp edges. 

Another layer is added that is more than  twice as big as the previous, also with the relu activation. I increased the size of the second layer as I wanted to try to force more features to be detected. With the assumption that simplified data removes noise, that means that the neurons should only learn the actual features of the drawing. 

Another pooling layer 

Then I dropout 10 percent of the hidden units to try and force learning upon the other 90 percent. This helps prevent overfitting. I have experimented with different values here, and it does not change the ending result too much. 

Then I flatten the model, so that the dimensionality is correct for the dense layer 

A densely connected layer that is double the amount of possible classes then connects the neurons followed by a 75 percent dropout that again randomly prevents overfitting. 

The last dense layer is then connected, this one containing 340 neurons, and with the  softmax activation function since we are dealing with a classification problem 

In [ ]:


#Constructing the actual CNN 
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(imheight, imwidth, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(num_classes*2, activation='relu'))
model.add(Dropout(0.75))

model.add(Dense(num_classes, activation='softmax'))
model.summary()

Here I define some callbacks and some rules, along with the learning rate and optimizer. 
Comments in code describe what I am doing. 


In [ ]:
def top_3_accuracy(x,y): 
    t3 = top_k_categorical_accuracy(x,y, 3)
    return t3

#Callbacks: 
#ReduceLROnPlateau reduces the learning rate when a parameter has stopped improving: In this case, if it hasn't seen significant improvement for 3 batches
# the learning rate is reduced by a factor 0.5 down to a minimum of 0.0001 learning rate
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                                   verbose=1, mode='autow', min_delta=0.005, cooldown=5, min_lr=0.0001)
#Stops traning when top 3 accuracy has stopped increasing for a total of 5 epochs 
earlystop = EarlyStopping(monitor='val_top_3_accuracy', mode='max', patience=5) 
#Setting the actual callbacks
callbacks = [reduceLROnPlat, earlystop]

#Compiling the mdel with cross entropy as performance measurer, adam optimizer, and with the top 3 accuracy metrics
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy', top_3_accuracy])

hists = []

#Training the model with the parameters set above. 
hist = model.fit(x=X_train, y=y_train,
          epochs = 40,
          batch_size=450,
          validation_data = (X_val, y_val),
          callbacks = callbacks,
          verbose = 1)

hists.append(hist)

- I found that including more epochs gave a gradual increase until a certain point where the increase was so small that it was not worth it. 
- Kaggle seems to crash and not continiue working at ~30 epochs. (entire page just vanishes, when refreshing everything is reset)  
- The max I have gotten is 78.68% using the adam optimizer, where other optimizers such as Adamax gave high initial accuracy, Adam ends with the highest. 
- Batch size  improves the initial results and the speed of the training significantly but results in a lower validation accuracy then if I choose a lower value. The model converges pretty quickly around 0.55
- Increasing the dropout rate from 0.5 to 0.75 seems to increase the accuracy quite a lot so I did the same thing with the first dropout. This, again, seems to increase the accuracy. This makes sense, as we force neurons to learn by deactivating some of them randomly. 
- The best result I could get with this relatively small network is between 77-79 %, depending on some of the above discussed hyperparameters


In [ ]:
from tensorflow.keras.metrics import categorical_accuracy

#creating a graph of the training: 
hist_df = pd.concat([pd.DataFrame(hist.history) for hist in hists], sort=True)
hist_df.index = np.arange(1, len(hist_df)+1)
fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
axs[0].plot(hist_df.val_top_3_accuracy, lw=5, label='Validation top 3 Accuracy', color="blue")
axs[0].plot(hist_df.top_3_accuracy, lw=5, label='Training top 3 Accuracy', color="red")
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].grid()
axs[0].legend(loc=0)

##Doing the same for loss
axs[1].plot(hist_df.val_loss, lw=5, label='Validation MLogLoss', color="blue")
axs[1].plot(hist_df.loss, lw=5, label='Training MLogLoss', color = "red")
axs[1].set_ylabel('MLogLoss')
axs[1].set_xlabel('Epoch')
axs[1].grid()
axs[1].legend(loc=0)
plt.show();

# Discussion 
Above we see the accuracy over time, and we can see that it actually still increases after epch 30. However Kaggle will not allow me to run more then 30 epochs before I time out or something like that, so I cannot train my final network. As expected, the loss is almost the inverse of what we see in the accuracy. 

If Kaggel ran the whoel network, I would have like to see what classes had the highest accruacy and what classes had the lowest. In this case, I speculate that more abstract and advanced classes would have a low accauracy, as people can interpret words differently. Also classes that might look alike, such as triangle and pyramid might get confused. 
Easy classes to identify would be classes with unique features such as the wall of china or the eiffel tower. 

As the data is converted to B/W that means that if any class that looks alike was only seperated by their colour, this network might have confused the two and therefore this accuracy would also have been low. 

